In [2]:
!pip install autocorrect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.8/622.8 KB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622380 sha256=dad7d4bb1100d6c84bffb31fa83403c317654e373767b0d57cc456511071c6a9
  Stored in directory: /root/.cache/pip/wheels/ab/0f/23/3c010c3fd877b962146e7765f9e9b08026cac8b035094c5750
Successfully built autocorrect


In [5]:
import pandas as pd
import os
import nltk
from sklearn.model_selection import GridSearchCV
import numpy as np
from nltk.corpus import stopwords
from autocorrect import Speller
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm
import spacy
import re
from nltk.tokenize import word_tokenize
nlp = spacy.load('en_core_web_sm')
import torch
import torch.nn as nn
from nltk.stem import WordNetLemmatizer
lemma = WordNetLemmatizer()
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from sklearn.svm import SVC
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from tqdm import tqdm


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
#new depressive data
data1 = pd.read_csv("data3.csv")

In [7]:
display(data1)

,Unnamed: 0,text,target
0,0,Ex Wife Threatening SuicideRecently I left my ...,1
1,3,i need helpjust help me im crying so hard,1
2,4,"I’m so lostHello, my name is Adam (16) and I’v...",1
3,5,Honetly idkI dont know what im even doing here...,1
4,6,[Trigger warning] Excuse for self inflicted bu...,1
...,...,...,...
4995,10163,Never trust anyoneMy wife has told me she isn'...,1
4996,10165,Someone to talk to would be nice.I've had feel...,1
4997,10166,I'm falling apartI've fucked up a lot in my li...,1
4998,10169,I can't do it anymore. I guess I'm just postin...,1


In [9]:
data1.drop("Unnamed: 0",axis = 1,inplace = True)
data1.rename(columns = {'target':'label'},inplace = True)

In [11]:
#old data
data2 = pd.read_csv("uncleaned_combined_txt.csv")

In [12]:
data2

,text,label
0,just had a real good moment. i missssssssss hi...,0
1,is reading manga http://plurk.com/p/mzp1e,0
2,@comeagainjen http://twitpic.com/2y2lx - http:...,0
3,@lapcat Need to send 'em to my accountant tomo...,0
4,ADD ME ON MYSPACE!!! myspace.com/LookThunder,0
...,...,...
12079,death sound appealingthought death thing bring...,1
12080,left relationship almost 5 years depression en...,1
12081,always wanted friendlife never anyone loved ca...,1
12082,deal someone avoids everythinghi close friend ...,1


In [13]:
#maintaining consistency
data2.rename(columns = {'target':'label'}, inplace = True)

In [14]:
data = pd.concat([data1,data2])

In [18]:
data['label'].value_counts()#balanced data

0    8807
1    8277
Name: label, dtype: int64

In [19]:
data.to_csv("Final_uncleaned_Balanced_Data")

In [24]:
data.reset_index(drop = True,inplace = True)

## Preprocessing

In [20]:
# preprocessing funciton created
def preprocessing(df):
    df1 = df.copy()
    df1['white_space_removed'] = 0
    df1['numbers_removed']=0
    df1['url_removed_data'] = 0
    df1['user_removed'] = 0
    df1['emoji_removed'] = 0
    df1['tokenized_data'] = 0
    df1['stopword_removed_data'] = 0
    df1['punct_removed_data'] = 0
    df1['spelling_checked_data'] = 0
    df1['lemma_data'] = 0
    stop_words = set(stopwords.words("english"))

#   lemmatizer = WordNetLemmatizer()
    spell = Speller(lang='en')


  # iterate over each row of dataset and preprocess data
    for i in tqdm(range(df1.shape[0])):
        # print(df1.text[i])
        # white space removal
        df1['white_space_removed'][i] = re.sub("\s+", " ", df1.text[i])
        df1['url_removed_data'][i] = re.sub('(www|http)\S+'," ",df1['white_space_removed'][i])
        df1['user_removed'][i] = re.sub('^@\S+'," ",df1['url_removed_data'][i])
        df1['numbers_removed'][i] = re.sub('\d+','',df1['user_removed'][i])
        emoji_removed = re.sub(r'\W+', ' ', df1['numbers_removed'][i].encode('ascii', 'ignore').decode('utf-8'), flags=re.UNICODE).strip()
        df1['emoji_removed'][i] = emoji_removed

        # lower casing and tokenization
        lower = df1['emoji_removed'][i].lower()
        tokenized_data = word_tokenize(lower)
        df1['tokenized_data'][i] = tokenized_data
        # print(tokenized_data)

        # remove stopwords
        stopword_removed_data = [x for x in tokenized_data if x not in stop_words]
        df1['stopword_removed_data'][i] = stopword_removed_data 
        # print(stopword_removed_data)
        
        
        # punctuation removal
        punct_removed_data = [x for x in stopword_removed_data if x.isalnum()]
        df1['punct_removed_data'][i] = punct_removed_data
        # print(punct_removed_data)
        

        spelling_checked_data = [spell(x) for x in punct_removed_data]
        df1['spelling_checked_data'][i] = spelling_checked_data
#         print(" ".join(spelling_checked_data))

        lemma_data = [lemma.lemmatize(x) for x in spelling_checked_data]
        df1['lemma_data'][i] = " ".join(lemma_data)

        # spelling checking

    return df1

In [25]:
preprocessed_data = preprocessing(data)

100%|██████████| 17084/17084 [38:17<00:00,  7.44it/s]


In [29]:
FINAL_DATA = pd.DataFrame(columns = ['embeddings','label'])

In [31]:
FINAL_DATA['embeddings'] = preprocessed_data['lemma_data']
FINAL_DATA['label'] = data['label']

In [32]:
FINAL_DATA

,embeddings,label
0,ex wife threatening suiciderecently left wife ...,1
1,need helpjust help im cry hard,1
2,im losthello name adam ive struggling year im ...,1
3,honestly idi dont know im even feel like nothi...,1
4,trigger warning excuse self inflicted burn kno...,1
...,...,...
17079,death sound appealingthought death thing bring...,1
17080,left relationship almost year depression engul...,1
17081,always wanted friendly never anyone loved care...,1
17082,deal someone avoids everything close friend le...,1


In [33]:
FINAL_DATA['label'].value_counts()

0    8807
1    8277
Name: label, dtype: int64

In [34]:
FINAL_DATA.to_csv("final_cleaned_data")